Nested Models

In [1]:
from pydantic import BaseModel

In [2]:
class Address(BaseModel):
    city: str
    zip_code: int

class User(BaseModel):
    id: int
    name: str
    address: Address

In [4]:
user = User(id=1,name='Tanmay',address = {'city':'Doon','zip_code':248001})
print(user)

id=1 name='Tanmay' address=Address(city='Doon', zip_code=248001)


Pydantic Fields : structure, constraints, defaults and specify validation rules

In [3]:
from pydantic import Field

String Constraints ( min_length=3, max_length=50 , regex=r"^[a-zA-Z]+$")

Integer/Float Constraints
 (   gt=0,       # greater than
    ge=18,      # greater or equal
    lt=100,     # less than
    le=120      # less or equal
)

In [4]:
class Item(BaseModel):
    name : str = Field(min_length=2, max_length=40)
    price : float = Field(gt=0, lt=1000)
    quant : int = Field(gt=0)

In [5]:
item = Item(name='Laptop', price=999.99, quant=5)
print(item)

name='Laptop' price=999.99 quant=5


Adding Documentation/Metadata

In [6]:
from pydantic import BaseModel, Field

class FieldUser(BaseModel):
    name: str = Field(description="The user's full name") 


field_user1 = FieldUser(name = "Tanmay")
print(field_user1)

name='Tanmay'


Custom Default Values

In [7]:
class userage(BaseModel):
    age : int = Field(description='User is atleast an adult',default=18,ge=18)
    male : bool = Field(default=True,validate_default=True)
user_age1 = userage()
print(user_age1)

age=18 male=True


Alias = Read with one name and Stored internally with another
We can access the data using our own field names

Alias = the  original key name the we are accepting to use as a alias

In [8]:
data = {
    "pkg_weight_kg": 4.5,
    "pkg_is_fragile": True,
    "pkg_dest": "Singapore"
}

In [9]:
class Package(BaseModel):
    weight: float = Field(alias="pkg_weight_kg")
    destination: str = Field(alias="pkg_dest")
    is_fragile: bool = Field(alias="pkg_is_fragile")

# using alias to used our custom defined field names

In [10]:
# unpack the dictionary
package = Package(**data)

In [11]:
print(package.weight)   
# instead of print(package.pkg_weight_kg)
print(package.destination) 

4.5
Singapore


In [12]:
# and the data can be exported with the org field names
print(package.model_dump(by_alias=True))

{'pkg_weight_kg': 4.5, 'pkg_dest': 'Singapore', 'pkg_is_fragile': True}


Validation and serialization allias : control how field names are interpreted on input vs output.

In [13]:
backend_data = {
    "book_title": "Pydantic Guide"}

In [14]:
class Book(BaseModel):
    title: str = Field(
        validation_alias="book_title",          # accepts as input
        serialization_alias="bookTitle"         # exported outputs name
    )

In [15]:
book = Book(**backend_data)

In [16]:
print(book.model_dump())  
print(book.model_dump(by_alias=True))

{'title': 'Pydantic Guide'}
{'bookTitle': 'Pydantic Guide'}


Required Fields : Useful when Adding metadata and Adding validation

not that useful

In [17]:
class User(BaseModel):
    name: str 

Both are required Feilds but using ... metadata and validation rules can be added

In [18]:
class User(BaseModel):
   email: str = Field(
    ...,
    min_length=5,
    description="Primary email address",
)

In [19]:
user = User(email="tanmay@example.com")
print(user)

email='tanmay@example.com'


Exclude Field : Sensitive data (passwords, tokens)

In [20]:
class User2(BaseModel):
    username: str
    password: str = Field(exclude=True)

In [21]:
user = User2(username="tanmay", password="securepassword123")
user.model_dump()


{'username': 'tanmay'}

Default Factory : Used to generate a new default value each time.

In [22]:
from datetime import datetime, timezone

In [23]:
class Box(BaseModel):
    items: list[str] = Field(default_factory=list)
    created_at: datetime = Field(
        default_factory=lambda: datetime.now(timezone.utc)
    )

In [25]:
box1 = Box()
print(box1)

items=[] created_at=datetime.datetime(2026, 1, 2, 12, 21, 46, 524726, tzinfo=datetime.timezone.utc)


In [26]:
box2 = Box(items=["apple", "banana"])
print(box2)

items=['apple', 'banana'] created_at=datetime.datetime(2026, 1, 2, 12, 22, 11, 786928, tzinfo=datetime.timezone.utc)
